In [17]:
#import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import TextVectorization

In [18]:
#load dataset

In [19]:
#image model (encoder)
encoder = EfficientNetB3(include_top=False,pooling='avg')
encoder.trainable = False
max_token = 10000
max_length = 30
vectorizer = TextVectorization(max_tokens=max_token, output_sequence_length=max_length+1)

In [ ]:
#feature pre processing
